In [1]:
import pandas as pd
import os
import soundfile as sf
import numpy as np
from pathlib import Path
import shutil
import base64
from scipy.io import wavfile
import warnings
import soundfile as sf
warnings.filterwarnings('ignore')

ROOT_DIR = '/home/aloradi/adversarial-robustness-for-sr'
DATA_DIR = f"{ROOT_DIR}{os.sep}data"
VPC_DIR = f"{DATA_DIR}{os.sep}vpc2025_official"
LIBRI_DIR =  DATA_DIR
os.chdir(ROOT_DIR)

In [2]:
def process_vpc_data(base_dir="data/vpc2025_official", 
                     num_samples=5, 
                     libri_dir="libriSpeech/train-clean-360/LibriSpeech/train-clean-360",
                     sep="|"):
    
    # List of directories to process (B4 and T10-2 do not follow the standard structure)
    target_dirs = ['B3', 'B4', 'B5', 'T10-2', 'T12-5', 'T25-1', 'T8-5']
    all_samples = []
    
    for dir_name in target_dirs:
        # Construct the path to combined_data.csv
        split_name = f"train-clean-360_{dir_name}"
        csv_path = os.path.join(base_dir, dir_name, "data", split_name, "combined_data.csv")
        
        if os.path.exists(csv_path):
            # Read the CSV file
            df = pd.read_csv(csv_path, sep=sep)
            
            # Sample rows
            if len(df) < num_samples:
                sampled_df = df
            else:
                sampled_df = df.sample(n=num_samples, random_state=42)
            
            # Add system name column
            sampled_df['system_name'] = dir_name
            
            all_samples.append(sampled_df)
        else:
            print(f"Warning: Could not find CSV file for {dir_name}")
    
    # Concatenate all samples
    combined_df = pd.concat(all_samples, ignore_index=True)
    
    # Create LibriSpeech path column
    combined_df['basename'] = combined_df['wav_path'].apply(lambda x: os.path.basename(x))
    combined_df['libri_path'] = combined_df['basename'].apply(
        lambda x: libri_dir + os.sep + x.split('-')[0] + os.sep + x.split('-')[1] + 
        os.sep  +  os.path.splitext(x)[0] + '.flac' 
    )
    
    return combined_df

In [3]:
def process_audio_file(file_path, root_dir):
    """Process audio file and return base64 encoded data"""
    # Read the WAV file
    file_path = root_dir + os.sep + file_path
    
    data, sample_rate = sf.read(file_path)

    # Convert to mono if stereo
    if len(data.shape) > 1:
        data = np.mean(data, axis=1)

    # Normalize audio data
    data = np.int16(data / np.max(np.abs(data)) * 32767)

    # Create a temporary file
    temp_path = f"temp_{os.path.basename(file_path)}"
    wavfile.write(temp_path, sample_rate, data)

    # Read and encode the temporary file
    with open(temp_path, 'rb') as audio_file:
        audio_data = base64.b64encode(audio_file.read()).decode('utf-8')

    # Clean up temporary file
    os.remove(temp_path)

    return f"data:audio/wav;base64,{audio_data}"

In [4]:
def create_html_report(df, output_dir="local/report_files", output_file="report.html"):
    os.makedirs(output_dir, exist_ok=True)
    
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>VPC Dataset Report</title>
        <style>
            table {{ border-collapse: collapse; width: 100%; }}
            th, td {{ border: 1px solid #ddd; padding: 12px; text-align: left; }}
            th {{ background-color: #f2f2f2; }}
            .audio-cell {{ width: 250px; }}
            .audio-player {{ width: 100%; }}
            .error {{ color: red; }}
        </style>
    </head>
    <body>
        <h1>VPC Dataset Report</h1>
        <p>Total samples: {len(df)}</p>
        <p>Systems analyzed: {df['system_name'].nunique()}</p>
        <table>
            <thead>
                <tr>
                    <th>Utterance Name</th>
                    <th>System Name</th>
                    <th>System Output</th>
                    <th>Original Sample</th>
                </tr>
            </thead>
            <tbody>
    """
    
    for _, row in df.iterrows():
        try:
            system_audio = process_audio_file(row['wav_path'], root_dir=VPC_DIR)
            original_audio = process_audio_file(row['libri_path'], root_dir=LIBRI_DIR)
            
            html_content += f"""
                <tr>
                    <td>{row['basename']}</td>
                    <td>{row['system_name']}</td>
                    <td class="audio-cell">
                    """
            
            if system_audio:
                html_content += f"""
                        <audio controls class="audio-player">
                            <source src="{system_audio}" type="audio/wav">
                            Your browser does not support the audio element.
                        </audio>
                        """
            else:
                html_content += '<p class="error">Error loading system audio</p>'
                
            html_content += '</td><td class="audio-cell">'
            
            if original_audio:
                html_content += f"""
                        <audio controls class="audio-player">
                            <source src="{original_audio}" type="audio/wav">
                            Your browser does not support the audio element.
                        </audio>
                        """
            else:
                html_content += '<p class="error">Error loading original audio</p>'
                
            html_content += '</td></tr>'
            
        except Exception as e:
            print(f"Error processing row {row['basename']}: {str(e)}")
            html_content += f"""
                <tr>
                    <td>{row['basename']}</td>
                    <td>{row['system_name']}</td>
                    <td colspan="2" class="error">Error processing audio files: {str(e)}</td>
                </tr>
            """
    
    html_content += """
            </tbody>
        </table>
    </body>
    </html>
    """
    
    output_path = os.path.join(output_dir, output_file)
    with open(output_path, 'w') as f:
        f.write(html_content)
    
    return output_path

In [6]:
# Process the data
processed_df = process_vpc_data()

# Create the HTML report
report_file = create_html_report(processed_df)

print(f"Report generated: {report_file}")
print("To view the report, start a local server:")
print("python -m http.server 8000")
print("Then open http://localhost:8000/local/report_files/report.html in your browser")

Report generated: locals/report_files/report.html
To view the report, start a local server:
python -m http.server 8000
Then open http://localhost:8000/local/report_files/report.html in your browser
